In [ ]:
# star_ko_data 에서 트로트가수 분류가 필요함
# series_id, 이름, 최신 앨범명, 발매 날짜, 장르(한 컬럼에 몰아서)
#


# 1차로 가수만 가지고 해보자

In [ ]:
# 중복 삭제
# DELETE a FROM star_album_genre a, star_album_genre b WHERE a.pk > b.pk AND a.series_id = b.series_id;

In [602]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime
from difflib import SequenceMatcher
import pymysql
from sqlalchemy import create_engine
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time

In [603]:
def db_connection(host_name='ds'):
    host_url = "db.ds.mycelebs.com"
    user_nm = "j_eungg"
    passwd = "wldms10529"
    port_num = 3306
    db_name = "star_ko"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8',
                           db=db_name, cursorclass=pymysql.cursors.DictCursor)
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태
    # db=db,
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    return conn, engine

## 1. VIBE key 값 (vibeCd) 수집

In [604]:
conn, engine = db_connection()
qry = "SELECT * FROM star_ko_data WHERE cd_category like '%가수%' and cd_is_use=1"
data = pd.read_sql(qry, conn)
data.head(2)

,cd_pk,series_id,cd_name,cd_category,cd_gender,cd_search_word,cd_solr_search,cd_real_name,cd_basic_info,cd_height,...,profile_url_rectangle_small,profile_url_diagonal,profile_url_square_medium,profile_url_square_small,profile_url_round_large,profile_url_round_medium,profile_url_round_small,profile_url_rectangle_large,profile_url_square_large,agency_idx
0,1,1,2AM,가수,남자,아이돌 | 보이그룹 | 콘서트 | 공연 | 음반 | 조권 | 창민 | 슬옹 | 진운...,"(content_all:(title:""2am"" AND ""기자"" ))",,None,NaN,...,https://all.image.mycelebs.com/1/1_1035@420.jpg,https://all.image.mycelebs.com/1/1_573@372.jpg,https://all.image.mycelebs.com/1/1_288@288.jpg,https://all.image.mycelebs.com/1/1_240@240.jpg,https://all.image.mycelebs.com/1/1_249@249.jpg,https://all.image.mycelebs.com/1/1_144@144.jpg,https://all.image.mycelebs.com/1/1_108@108.jpg,https://all.image.mycelebs.com/1/1_1035@738.jpg,https://all.image.mycelebs.com/1/1_420@420.jpg,NaN
1,2,2,투애니원,가수,여자,"걸그룹 | 콘서트 | 공연 | 음반 | 씨엘 | 봄 | 다라 | 민지 | CL ""2...","(content_all:((title:""2ne1"" title:""투애니원"" title...",2NE1,None,NaN,...,https://all.image.mycelebs.com/2/2_1035@420.jpg,https://all.image.mycelebs.com/2/2_573@372.jpg,https://all.image.mycelebs.com/2/2_288@288.jpg,https://all.image.mycelebs.com/2/2_240@240.jpg,https://all.image.mycelebs.com/2/2_249@249.jpg,https://all.image.mycelebs.com/2/2_144@144.jpg,https://all.image.mycelebs.com/2/2_108@108.jpg,https://all.image.mycelebs.com/2/2_1035@738.jpg,https://all.image.mycelebs.com/2/2_420@420.jpg,161236.0


In [605]:
data_list = data['series_id'].to_list()
print("star_ko_data 가수 수 :", len(data_list))

star_ko_data 가수 수 : 5819


In [606]:
qry = f'SELECT * FROM star_ko_profile_url WHERE naver_url is not NULL AND naver_url != \'0\' and naver_url != \'x\''
profileData = pd.read_sql(qry, con=conn)
profileData_list = profileData['series_id'].to_list()
print("star_ko_profile_url 네이버 프로필 수집된 개체 수 :", len(profileData_list))

star_ko_profile_url 네이버 프로필 수집된 개체 수 : 10138


In [591]:
# 수집할 대상

data_ = []

for p in profileData_list :
    if p in data_list :
        data_.append(p)
        
print("가수 중에서 네이버 프로필 있는 개체 수 :", len(data_))

가수 중에서 네이버 프로필 있는 개체 수 : 3763


In [592]:
genreData_list = []

conn, engine = db_connection()

# qry = f'SELECT * FROM star_album_genre WHERE vibeCd is not NULL'
qry = f'SELECT * FROM star_album_genre'
genreData = pd.read_sql(qry, con=conn)
genreData_list = genreData['series_id'].to_list()

In [593]:
# 최종 대상
# 수집 대상 중에 star_album_genre 에 없는 사람

final = []

for idx in data_ :
    if idx not in genreData_list :
        final.append(idx) 

In [594]:
len(final)

1977

In [595]:
conn.close()

In [598]:
conn, engine = db_connection()

qry = f'SELECT * FROM star_album_genre WHERE series_id >= 1823 and series_id <= 2304'
final = pd.read_sql(qry, con=conn)
final = final['series_id'].to_list()

In [600]:
len(final)

102

In [601]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

number = int(input("vibeCd 수집할 개체 수 입력 :"))
ex = final[0:number]

for i in tqdm(ex) :
    conn, engine = db_connection()
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    #vibeCd = ''
    
    qry = f'SELECT * FROM star_ko_profile_url WHERE series_id={i}'
    profileData = pd.read_sql(qry, con=conn)
    
    series_id = name = naver_url = vibe_url = ''
    
    series_id = profileData['series_id'][0]
    name = profileData['name'][0]
    naver_url = profileData['naver_url'][0]
    
    naver_html = requests.get(naver_url).text
    time.sleep(0.5)
    naver_soup = BeautifulSoup(naver_html, 'html.parser')
    time.sleep(0.5)
    
    # VIBE url 찾기
    try :
        tit = naver_soup.select('a[class=tit]')

        for t in tit :
            if 'vibe' in t.get('href') :
                vibe_url = t.get('href')

        driver.get(vibe_url)
        time.sleep(3)
        try :
            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click()
        except :
            pass

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        #time.sleep(3)

        # VIBE key 값
        #vibeCd = ''
        vibeCd = soup.select('a[class=link_sub_title]')[0].get('href')
        vibeCd = re.findall("\d+", vibeCd)[0]

        qry = f"INSERT into star_album_genre (series_id, vibeCd, name) VALUES ({series_id}, {vibeCd}, '{name}')"
        cursor.execute(qry)
        conn.commit()
        print("[INFO]", name, " insert 완료")
        
    except :
        print("VIBE url NONE")
        
        qry = f"INSERT into star_album_genre (series_id, name) VALUES ({series_id}, '{name}')"
        cursor.execute(qry)
        conn.commit()

    
conn.close()
driver.close()



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


vibeCd 수집할 개체 수 입력 :102


  1%|          | 1/102 [00:08<14:25,  8.57s/it]

[INFO] 임현식  insert 완료


  2%|▏         | 2/102 [00:14<12:05,  7.26s/it]

...

[INFO] 이창민  insert 완료


 99%|█████████▉| 101/102 [09:07<00:04,  4.60s/it]

VIBE url NONE


100%|██████████| 102/102 [09:13<00:00,  5.43s/it]

[INFO] 찬성  insert 완료


## 1.1 수기 매핑 (중복값 & null 값)

In [607]:
conn, engine = db_connection()

qry = f'SELECT * FROM star_album_genre WHERE vibeCd is NULL'
nullData = pd.read_sql(qry, con=conn)
nullData_list = nullData['series_id'].to_list()

In [608]:
qry = f'SELECT * FROM star_album_genre GROUP BY vibeCd HAVING COUNT(vibeCd) > 1'
dupData = pd.read_sql(qry, con=conn)
dupData_list = dupData['series_id'].to_list()

In [609]:
handmapping = []

handmapping = nullData_list + dupData_list
print("nullData_list : ", len(nullData_list), "dupData_list : ", len(dupData_list), "합 : ", len(handmapping))

nullData_list :  199 dupData_list :  272 합 :  471


In [610]:
conn.close()

In [584]:
conn, engine = db_connection()
cursor = conn.cursor(pymysql.cursors.DictCursor)

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get('https://vibe.naver.com/search?query=')

naver_driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

for idx in tqdm(handmapping) :
    qry = f'SELECT series_id, cd_name, cd_real_name, cd_category, cd_gender, cd_solr_search, cd_height, cd_weight, cd_birth, cd_agency, cd_debut, cd_academic, cd_career, cd_award FROM star_ko_data WHERE series_id={idx}'
    result = pd.read_sql(qry, conn)
    display(result)
    
    qry2 = f'SELECT * FROM star_ko_profile_url WHERE series_id={idx}'
    result2 = pd.read_sql(qry2, conn)
    series_id = result2['series_id'][0]
    name = result2['name'][0]
    naver = result2['naver_url'][0]
    
    naver_driver.get(naver)
    
    driver.get(f'https://vibe.naver.com/search?query={name}')
    
    print(naver)
    
    try :
        driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click()
    except :
        pass
    
    vibe = input("vibeCd를 입력하세요. ")
    
    if vibe == '' :
        vibeCd = '"x"'
    else :
        vibeCd = vibe
        
    updateQRY = f'UPDATE star_album_genre SET vibeCd={vibeCd} WHERE series_id={series_id}'
    cursor.execute(updateQRY)
    conn.commit()



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
  0%|          | 0/467 [00:00<?, ?it/s]

,series_id,cd_name,cd_real_name,cd_category,cd_gender,cd_solr_search,cd_height,cd_weight,cd_birth,cd_agency,cd_debut,cd_academic,cd_career,cd_award
0,1812,임선영,,가수,여자,"(content_all:(""Y미디어 임선영""~10 OR ""가수 임선영""~10))",174,49,1983-02-23,Y미디어,2001년 거북이 1집 앨범 'Go Boogie',수원대학교 연극영화학과,,


vibeCd를 입력하세요. 3067344


  0%|          | 1/467 [00:28<3:45:12, 29.00s/it]

,series_id,cd_name,cd_real_name,cd_category,cd_gender,cd_solr_search,cd_height,cd_weight,cd_birth,cd_agency,cd_debut,cd_academic,cd_career,cd_award
0,1814,임세준,,가수,남자,"(content_all:(title:""임세준"" AND ""기자"" AND (""가수 임세...",175,68,1989-07-18,도우즈 레코즈,2012년 싱글 앨범 'To Love Somebody',서울예술대학교 실용음악과,,


...

vibeCd를 입력하세요. 214557


  1%|▏         | 6/467 [01:20<1:30:00, 11.72s/it]

,series_id,cd_name,cd_real_name,cd_category,cd_gender,cd_solr_search,cd_height,cd_weight,cd_birth,cd_agency,cd_debut,cd_academic,cd_career,cd_award
0,1830,장석현,,"가수,배우",남자,"(content_all:(""영화배우 장석현""~10 OR ""배우 장석현""~10 OR ...",178,70,1980-10-26,EPP미디어,1998년 샵 1집 앨범 'The Sharp',동아방송대학 방송연예과,,


  1%|▏         | 6/467 [02:18<2:57:25, 23.09s/it]


KeyboardInterrupt: Interrupted by user

## 2. 장르 수집

In [573]:
conn, engine = db_connection()

qry = f'SELECT * FROM star_album_genre WHERE vibeCd is not NULL and vibeCd != \'x\''
data = pd.read_sql(qry, con=conn)
data.head(3)

,pk,series_id,vibeCd,name,new_albumName,release_date,genre,regist_date,update_date
0,1,1,110359,2AM,None,None,None,2021-09-07 05:55:52,NaT
1,2,2,117564,2NE1,None,None,None,2021-09-07 05:55:57,NaT
2,3,3,112090,2PM,None,None,None,2021-09-07 05:56:02,NaT


In [570]:
len(data)

1097

In [572]:
data_list = data['vibeCd'].to_list()

In [ ]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

number = int(input("장르 분류할 개체 수 입력 :"))
ex = data_list[0:number]

album_url = []
genre_list = []

for i in tqdm(ex) :
    conn, engine = db_connection()
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    vibe_url = 'https://vibe.naver.com/artist/'+i
    
    driver.get(vibe_url)
    try : 
        driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click() # 팝업창 끄기
    except :
        pass
    
    while True :    
        try :    
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤 끝까지 내리기
            time.sleep(0.5)

            driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/a').click() # 더보기 클릭
            time.sleep(0.5)
        except :
            break
            
    vibe_html = driver.page_source
    vibe_soup = BeautifulSoup(vibe_html, 'html.parser')
    
    # 앨범 리스트
    album_list = vibe_soup.select('a[class=title]')

    for i in album_list :
        href = i.get('href')
        href = 'https://vibe.naver.com' + href
        album_url.append(href)
        
        
    # 최신 앨범명 & 발매 날짜
    newURL = album_url[0]

    driver.get(newURL)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 앨범명
    title = soup.select('span[class=title]')[0].text
    title = re.sub("\!|\'|\?", "", title)  # 특수문자 제거
    # 발매 날짜
    relese_date = soup.select('span[class=item]')[0].text
    relese_date = relese_date.replace(' ', '')
        
    # 모든 장르
    for j in album_url :
        driver.get(j)
        html_ = driver.page_source
        soup_ = BeautifulSoup(html_, 'html.parser')
        
        genre = soup_.select('span[class=item]')[1].text
        genre = genre.replace(' ', '')
        
        genre_list.append(genre)
        
    genre_list = list(set(genre_list)) # 중복 제거
    
    # 사업자 등록 번호 제거
    for a in genre_list[::-1] :
        if a.find('사업자') > -1 :
            genre_list.remove(a)
            
    # 리스트 문자열로 합치기
    genre_str = '|'.join(genre_list)

    insertQRY = f"INSERT into star_album_genre (series_id, vibeCd, name, new_albumName, release_date, genre) \
                VALUES ({series_id}, {vibeCd}, '{name}', '{title}', '{relese_date}', '{genre_str}')"
    cursor.execute(insertQRY)
    conn.commit()
    
conn.close()
driver.close()


In [103]:
### vibeCd & 장르 한꺼번에 수집하는 코드

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

number = int(input("장르 분류할 개체 수 입력 :"))
ex = data_list[0:number]

album_url = []
genre_list = []

for i in tqdm(ex) :
    conn, engine = db_connection()
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    qry = f'SELECT * FROM star_ko_profile_url WHERE series_id={i}'
    profileData = pd.read_sql(qry, con=conn)
    
    series_id = profileData['series_id'][0]
    name = profileData['name'][0]
    naver_url = profileData['naver_url'][0]
    
    naver_html = requests.get(naver_url).text
    naver_soup = BeautifulSoup(naver_html, 'html.parser')
    
    # 앨범 상세보기 > 바이브
    vibe_url = naver_soup.select('a[class=bt_more]')[0].get('href')
    vibeCd = re.findall("\d+", vibe_url)[0]
    
    driver.get(vibe_url)
    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click() # 팝업창 끄기
    
    while True :    
        try :    
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤 끝까지 내리기
            time.sleep(1)

            driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/a').click() # 더보기 클릭
            time.sleep(1)

        except :
            break
            
    vibe_html = driver.page_source
    vibe_soup = BeautifulSoup(vibe_html, 'html.parser')
    
    # 앨범 리스트
    album_list = vibe_soup.select('a[class=title]')

    for i in album_list :
        href = i.get('href')
        href = 'https://vibe.naver.com' + href
        album_url.append(href)
        
        
    # 최신 앨범명 & 발매 날짜
    newURL = album_url[0]

    driver.get(newURL)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 앨범명
    title = soup.select('span[class=title]')[0].text
    title = re.sub("\!|\'|\?", "", title)  # 특수문자 제거
    # 발매 날짜
    relese_date = soup.select('span[class=item]')[0].text
    relese_date = relese_date.replace(' ', '')
        
    # 모든 장르
    for j in album_url :
        driver.get(j)
        html_ = driver.page_source
        soup_ = BeautifulSoup(html_, 'html.parser')
        
        genre = soup_.select('span[class=item]')[1].text
        genre = genre.replace(' ', '')
        
        genre_list.append(genre)
        
    genre_list = list(set(genre_list)) # 중복 제거
    
    # 사업자 등록 번호 제거
    for a in genre_list[::-1] :
        if a.find('사업자') > -1 :
            genre_list.remove(a)
            
    # 리스트 문자열로 합치기
    genre_str = '|'.join(genre_list)

    insertQRY = f"INSERT into star_album_genre (series_id, vibeCd, name, new_albumName, release_date, genre) \
                VALUES ({series_id}, {vibeCd}, '{name}', '{title}', '{relese_date}', '{genre_str}')"
    cursor.execute(insertQRY)
    conn.commit()
    
conn.close()
driver.close()




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


장르 분류할 개체 수 입력 :1


  0%|          | 0/1 [00:35<?, ?it/s]


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 's Talk', '2014.10.30', '응원가|발라드,알앤비/어반|댄스|캐롤|영' at line 1")

In [188]:
naver_url = 'https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query=%EC%83%A4%EC%9D%B4%EB%8B%88%ED%83%9C%EB%AF%BC&os=154915&ie=utf8&key=PeopleService'

naver_html = requests.get(naver_url).text
naver_soup = BeautifulSoup(naver_html, 'html.parser')

tit = naver_soup.select('a[class=tit]')

for t in tit :
    if 'vibe' in t.get('href') :
        vibe_url = t.get('href')
        
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

driver.get(vibe_url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# VIBE key 값
vibeCd = soup.select('a[class=link_sub_title]')[0].get('href')
vibeCd = re.findall("\d+", vibeCd)[0]

qry = f"INSERT into star_album_genre (series_id, vibeCd, name) VALUES ({series_id}, {vibeCd}, '{name}')"

In [451]:
conn.close()

In [456]:
conn, engine = db_connection()
qry = 'SELECT * FROM star_album_genre_copy_ GROUP BY vibeCd HAVING COUNT(vibeCd) > 1'
df = pd.read_sql(qry, con=conn)
df.head()
# df_list = df['series_id'].to_list()

,pk,series_id,vibeCd,name,new_albumName,release_date,genre,regist_date,update_date
0,2423,8052008,101699,윙크,None,None,None,2021-09-06 17:11:16,None
1,3456,16388181,102088,다빈크,None,None,None,2021-09-06 18:46:49,None
2,1225,3244,104494,마이티 마우스,None,None,None,2021-09-06 14:56:23,None
3,178,398,10476,배치기,None,None,None,2021-09-06 13:12:13,None
4,181,406,106,백지영,None,None,None,2021-09-06 13:12:29,None


In [462]:
df.shape

(477, 9)

In [459]:
df_list = df['vibeCd'].to_list()

In [460]:
len(df_list)

477

In [461]:
df_list

['101699',
 '102088',
 '104494',
 '10476',
 '106',
 '107338',
 '1076039',
 '107746',
 '10793',
 '10838',
 '1090381',
 '1090664',
 '109320',
 '109713',
 '11001',
 '110111',
 '110142',
 '11156',
 '112',
 '112090',
 '112268',
 '114411',
 '115',
 '115457',
 '115672',
 '116436',
 '11691',
 '116932',
 '118106',
 '11857',
 '11925',
 '119353',
 '119647',
 '1200405',
 '1200595',
 '120130',
 '121224',
 '121611',
 '121722',
 '122084',
 '122134',
 '122136',
 '1229',
 '1231',
 '123222',
 '12435',
 '124674',
 '125352',
 '12536',
 '12543',
 '125617',
 '126049',
 '127297',
 '128608',
 '129',
 '1304',
 '131660',
 '133892',
 '13542',
 '135502',
 '135666',
 '135669',
 '136625',
 '137277',
 '138432',
 '138437',
 '138458',
 '1385',
 '138775',
 '138794',
 '139059',
 '14044',
 '140917',
 '14139',
 '14227',
 '142945',
 '143112',
 '143179',
 '143404',
 '144',
 '1441118',
 '14452',
 '1466571',
 '14848',
 '148483',
 '1485228',
 '1485262',
 '148769',
 '1490807',
 '152583',
 '15292',
 '1541899',
 '154543',
 '15462

In [466]:
print(", ".join(df_list), end='\n\n')

101699, 102088, 104494, 10476, 106, 107338, 1076039, 107746, 10793, 10838, 1090381, 1090664, 109320, 109713, 11001, 110111, 110142, 11156, 112, 112090, 112268, 114411, 115, 115457, 115672, 116436, 11691, 116932, 118106, 11857, 11925, 119353, 119647, 1200405, 1200595, 120130, 121224, 121611, 121722, 122084, 122134, 122136, 1229, 1231, 123222, 12435, 124674, 125352, 12536, 12543, 125617, 126049, 127297, 128608, 129, 1304, 131660, 133892, 13542, 135502, 135666, 135669, 136625, 137277, 138432, 138437, 138458, 1385, 138775, 138794, 139059, 14044, 140917, 14139, 14227, 142945, 143112, 143179, 143404, 144, 1441118, 14452, 1466571, 14848, 148483, 1485228, 1485262, 148769, 1490807, 152583, 15292, 1541899, 154543, 154626, 154659, 15466, 155052, 155310, 155629, 156, 156525, 156842, 15838, 160, 1619112, 1620399, 1620438, 1622092, 1622228, 1622620, 1629875, 1629941, 1630433, 1630881, 1649093, 1650044, 166971, 167166, 1677995, 1687805, 16975, 170717, 171334, 171437, 173081, 173672, 17392, 1748968, 1

In [470]:
conn, engine = db_connection()
qry = 'SELECT * FROM star_album_genre_copy_'
alldf = pd.read_sql(qry, con=conn)
alldf.head()

,pk,series_id,vibeCd,name,new_albumName,release_date,genre,regist_date,update_date
0,23,2,117564,2NE1,None,None,None,2021-09-06 09:31:57,NaT
1,30,9,208477,EXO-K,None,None,None,2021-09-06 09:32:21,NaT
2,34,13,1234,god,None,None,None,2021-09-06 09:32:35,NaT
3,38,15,143596,JYJ,None,None,None,2021-09-06 09:40:08,NaT
4,41,18,9346,MC몽,None,None,None,2021-09-06 09:40:24,NaT


In [471]:
len(alldf)

2331

In [521]:
conn.close()